# pandas ecosystem 2019

@datapythonista

In [40]:
data = [1, 2, 3, 4]

In [20]:
sum(data * 2)

20

In [21]:
data * 2

[1, 2, 3, 4, 1, 2, 3, 4]

In [44]:
[item * 2 for item in data]

[2, 4, 6, 8]

In [31]:
class FancyList:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return str(self.value)

    def __mul__(self, other):
        return List([item * other for item in self.value])

    def __rmul__(self, other):
        return self * other

    def sum(self):
        return sum(self.value)

In [32]:
fancy_obj = FancyList(data)

fancy_obj * 2

[2, 4, 6, 8]

In [34]:
fancy_obj.sum()

10

pyobject...

https://www.python.org/dev/peps/pep-3118/

In [42]:
import array

array.array('I', [1, 2, 3, 4])

array('I', [1, 2, 3, 4])

In [43]:
import static_frame

static_frame.Frame

static_frame.core.frame.Frame